In [1]:
import sys
sys.path.append("..")
from globals import *

from support import read_and_save_cleaned_data,extract_vanillaNN_features,extract_expert_features,extract_dollar_features

with open('../data/participants.pickle', 'rb') as handle:
    _, pIDs_personalized,pIDs_standardized,pIDs_analogous,pIDs_all = pickle.load(handle)

Was not able to open the participant IDs


# segment gestures and save raw data to files

- ```pIDs_exptr_only``` = all participants who performed the ``personalized'' gestures
- ```pIDs_standardized``` = all participants who performed the ``standardized'' gestures
- ```pIDs_rotate``` = all participants who performed the ``analogous'' gestures
- ```pIDs_all``` = all participants involved in this study 

- save as CSV

https://physionet.org/about/publish/#author_guidelines

In [2]:
# # EXTRACT AUTOMATED SEGMENTATION TIMES FROM CSV FILE
# for pID in pIDs_all:
#     # read in the segmentation data
#     activity_cutoffs = pd.read_csv('C:/Users/yamag/Box/Research/Data/UW biosignal gesture study/segmentation/segmentation_csv_with_rehab/'+pID+".csv")
#     activity_cutoffs = activity_cutoffs[activity_cutoffs["remove"] == False].reset_index(drop=True)

#     # read in the filenames
#     pID_data = pd.read_csv('D:/Research/2024_UIST_dataset/upload/fnames/'+pID+"_fnames.csv",index_col=0)
#     # display(pID_data.head())
#     fnames = pID_data['fname_EMG'].values
#     start_times = []
#     stop_times = []
#     for filename in fnames:
#         activity_cutoffs_file = activity_cutoffs[activity_cutoffs['file'] == filename].reset_index(drop=True)
#         start = activity_cutoffs_file[activity_cutoffs_file["start"] == True].reset_index()
#         stop = activity_cutoffs_file[activity_cutoffs_file["stop"] == True].reset_index()

#         # if there are no start or end times, that means no segmentation should occur
#         # if override (or whatever you end up calling it) then override that with any other value
#         if len(start) == 0:
#             start_time = np.nan
#         # elif len(np.where(start["override"] == True)[0] > 0):
#         #     start_time = start.loc[np.where(start["override"] == True)[0][0], "time_start_window"]
#         else:
#             start_time = start["time_start_window"].min()

#         if len(stop) == 0:
#             stop_time = np.nan
#         # elif len(np.where(stop["override"] == True)[0] > 0):
#         #     stop_time = stop.loc[np.where(stop["override"] == True)[0][0], "time_end_window"]
#         else:
#             stop_time = stop["time_end_window"].max()
#         start_times.append(start_time)
#         stop_times.append(stop_time)
#     pID_data["start_times"] = start_times
#     pID_data["stop_times"] = stop_times
#     pID_data.to_csv('D:/Research/2024_UIST_dataset/fnames_with-segmentation/'+pID+"_fnames.csv")

In [3]:
# IMU_colnames = []
# imu_types = ["ax","ay","az","vx","vy","vz"]
# # for ix in range(72):
# for iy in [1,2,3,4,5,6,7,8,9,11,13,15]:
#     for iz in range(len(imu_types)):
#         # print("IMU"+str(int(iy))+"_"+imu_types[iz])
#         IMU_colnames.append("IMU"+str(int(iy))+"_"+imu_types[iz])

In [4]:
# # SEGMENT THE ACTUAL DATA

# fs_EMG = 2000
# fs_IMU = 148.148
# for pID in pIDs_all:
#     # read in the filenames
#     pID_data = pd.read_csv('D:/Research/2024_UIST_dataset/fnames_with-segmentation/'+pID+"_fnames.csv",index_col=0)

#     # EMG_fnames = pID_data['fname_EMG'].values
#     # IMU_fnames = pID_data['fname_IMU'].values
#     path = global_path + ('with disability/' if pID[1]=='1' else 'without disability/') + pID + '/'
#     for ix,row in pID_data.iterrows():
#         EMG_fname = row.fname_EMG
#         IMU_fname = row.fname_IMU
#         start_time = row.start_times
#         stop_time = row.stop_times
#         df_EMG = pd.read_csv(path+EMG_fname,sep="\t",names=['EMG1','EMG2','EMG3','EMG4','EMG5',
#                                                 'EMG6','EMG7','EMG8','EMG9','EMG10',
#                                                 'EMG11','EMG12','EMG13','EMG14','EMG15',
#                                                 'EMG16'],header=None)
#         # check if EMG activity is doubled
#         if df_EMG["EMG2"].values[2]==df_EMG["EMG2"].values[3]:
#             df_EMG = df_EMG.iloc[::2, :] # participant has redundant EMG activity
#             start_time = start_time/2
#             stop_time = stop_time/2

#         # check if start and stop are nans 
#         if math.isnan(start_time):
#             start_time = 0
#         if math.isnan(stop_time):
#             stop_time = len(df_EMG)/fs_EMG

#         df_EMG = df_EMG.iloc[int(np.floor(start_time*fs_EMG)):int(np.ceil(stop_time*fs_EMG))]
#         df_EMG.reset_index(inplace=True,drop=True)
#         # print("_".join([EMG_fname.split("_")[i] for i in [1,3,4,5]]))
#         if not os.path.isdir('D:/Research/2024_UIST_dataset/upload/segmented_raw_data/'+pID):
#             os.mkdir('D:/Research/2024_UIST_dataset/upload/segmented_raw_data/'+pID)
#         df_EMG.to_csv('D:/Research/2024_UIST_dataset/upload/segmented_raw_data/'+pID+'/'+EMG_fname[:-4]+".csv")

#         df_IMU = pd.read_csv(path+IMU_fname,sep="\t",header=None)
#         # pull out relevant columns 
#         sensors = [1,2,3,4,5,6,7,8,9,11,13,15]
#         cols = np.hstack([range(int((ix-1)*9),int(ix*9-3)) for ix in sensors])
#         df_IMU = df_IMU[cols]
#         df_IMU = df_IMU.iloc[int(np.floor(start_time*fs_IMU)):int(np.ceil(stop_time*fs_IMU))]
#         df_IMU.reset_index(inplace=True,drop=True)
#         df_IMU = df_IMU.set_axis([IMU_colnames],axis=1)
#         df_IMU.to_csv('D:/Research/2024_UIST_dataset/upload/segmented_raw_data/'+pID+'/'+IMU_fname[:-4]+".csv")

# filter EMG data for $M and vanilla NN

In [3]:
for pID in pIDs_all:
    read_and_save_cleaned_data(pID,extract_path)

In [10]:
for pID in pIDs_all:
    extract_dollar_features(pID)
    # break;

# interpolate, stack, and save data for vanilla NN

In [7]:
# for pID in pIDs_all:
#     extract_vanillaNN_features(pID)

# extract expert_features

In [8]:
# for pID in pIDs_all:
#     extract_expert_features(pID)

# delete old folders

In [9]:
# import shutil
# models_path = "D:\\Research\\2024_UIST_dataset\\models\\"
# try:
#     with open('data/participants.pickle', 'rb') as handle:
#         _, pIDs_personalized,pIDs_standardized,pIDs_analogous,pIDs_all = pickle.load(handle)
# except:
#     print("Was not able to open the participant IDs")
#     pass;

# p = models_path
# for pID in pIDs_all:
#     folders = glob.glob(p+pID+"/rotate/")
#     print(folders)
#     # for folder in folders:
#     #     f = folder.split("\\")[-1]
#     #     if f=="vanillaNN_files":# or f =="rotate":
#     #         shutil.rmtree(folder)
#     #     else:
#     #         pass;
